In [23]:
import pandas as pd

df_fw = pd.read_csv("data/Davos_Apartments.csv")   # Ferienwohnungen
df_hotel = pd.read_csv("data/Davos_Hotels.csv")    # Hotels


In [24]:
df_fw["type"] = "ferienwohnung"
df_hotel["type"] = "hotel"


In [25]:
df_combined = pd.concat([df_fw, df_hotel], ignore_index=True)


In [26]:
df_combined.to_csv("data/Davos_combined.csv", index=False)


In [27]:
# (pandas is already imported in another cell)
df = pd.read_csv("data/Davos_combined.csv")

# --- Skilift-Spalte zusammenführen (wenn vorhanden) ---
if "distance_skilift_raw" in df.columns:
	if "distance_skilift" in df.columns:
		df["distance_skilift"] = df["distance_skilift"].combine_first(df["distance_skilift_raw"])
	else:
		df["distance_skilift"] = df["distance_skilift_raw"]
	df = df.drop(columns=["distance_skilift_raw"])

# --- Zentrum-Spalte zusammenführen (unterstütze Varianten für den Raw-Name) ---
raw_zentrum = next((c for c in ("distance_zentrum_raw", "distance_centre_raw") if c in df.columns), None)
if raw_zentrum is not None:
	if "distance_zentrum" in df.columns:
		df["distance_zentrum"] = df["distance_zentrum"].combine_first(df[raw_zentrum])
	else:
		df["distance_zentrum"] = df[raw_zentrum]
	df = df.drop(columns=[raw_zentrum])

# --- Zimmergrösse zusammenführen ---
if "size_raw" in df.columns:
	if "zimmer_grösse" in df.columns:
		df["zimmer_grösse"] = df["zimmer_grösse"].combine_first(df["size_raw"])
	else:
		df["zimmer_grösse"] = df["size_raw"]
	df = df.drop(columns=["size_raw"])

# Speichern (lokaler data-Ordner)
df.to_csv("data/Davos_combined_clean.csv", index=False)

df.head()


,web_scraper_order,web_scraper_start_url,price_raw,type,distance_zentrum,distance_skilift,zimmer_grösse
0,1763925941-1,https://www.booking.com/searchresults.de.html?...,CHF 411,ferienwohnung,"Zentrum: 0,8 km",100 m vom Skilift entfernt,Apartment mit Blick auf die Berge
1,1763925941-2,https://www.booking.com/searchresults.de.html?...,CHF 134,ferienwohnung,"Zentrum: 0,9 km",600 m vom Skilift entfernt,Comfort Studio (2 Erwachsene)
2,1763925941-3,https://www.booking.com/searchresults.de.html?...,CHF 120,ferienwohnung,"Zentrum: 1,4 km","1,2 km vom Skilift entfernt",Zweibettzimmer
3,1763925941-4,https://www.booking.com/searchresults.de.html?...,CHF 199,ferienwohnung,"Zentrum: 0,8 km",300 m vom Skilift entfernt,Standard Doppelzimmer
4,1763925941-5,https://www.booking.com/searchresults.de.html?...,CHF 285,ferienwohnung,"Zentrum: 1,2 km",100 m vom Skilift entfernt,Executive Doppelzimmer


In [28]:
df["distance_zentrum"] = (
    df["distance_zentrum"]
    .astype(str)
    .str.replace("Zentrum", "", case=False, regex=False)  # entfernt "Zentrum" & "zentrum"
    .str.replace(":", "", regex=False)                    # optional: ":" entfernen
    .str.strip()                                          # Leerzeichen trimmen
)


In [29]:
# normalize NBSPs and remove thousand separators from price_raw, then save
df["price_raw"] = (
    df["price_raw"]
    .astype(str)
    .str.replace("\u00A0", " ", regex=False)         # normalise non-breaking spaces
    .str.replace(r"(?<=\d)\.(?=\d)", "", regex=True) # remove dots between digits (thousand separators)
    .str.strip()
)

# save to the cleaned combined CSV (don't use undefined variable `p`)
df.to_csv("data/Davos_combined_clean.csv", index=False)

# show a quick preview
df["price_raw"].head(10)

0     CHF 411
1     CHF 134
2     CHF 120
3     CHF 199
4     CHF 285
5     CHF 165
6     CHF 198
7    CHF 1170
8     CHF 464
9     CHF 679
Name: price_raw, dtype: object

In [30]:
# entferne die Wörter "vom Skilift entfernt" (case-insensitive, inkl. NBSP-Normalisierung) und speichere
if "distance_skilift" in df.columns:
    df["distance_skilift"] = (
        df["distance_skilift"]
        .astype(str)
        .str.replace("\u00A0", " ", regex=False)                       # NBSP normalisieren
        .str.replace(r"\s*vom\s*skilift\s*entfernt", "", regex=True, case=False)
        .str.strip()
    )
    # save to the cleaned combined CSV (use the same path used elsewhere)
    df.to_csv("data/Davos_combined_clean.csv", index=False)

# schnelle Vorschau
df["distance_skilift"].head(10)

0     100 m
1     600 m
2    1,2 km
3     300 m
4     100 m
5     500 m
6     650 m
7     550 m
8     200 m
9     550 m
Name: distance_skilift, dtype: object

In [31]:
# "km" in der Spalte "distance_skilift" in Meter umwandeln (nur falls als String vorhanden)
def km_to_meter(val):
    if isinstance(val, str) and "km" in val:
        try:
            return int(float(val.replace("km", "").replace(",", ".").strip()) * 1000)
        except ValueError:
            return val
    return val

df["distance_skilift"] = df["distance_skilift"].apply(km_to_meter)

In [32]:
df["distance_skilift"].head(10)

0    100 m
1    600 m
2     1200
3    300 m
4    100 m
5    500 m
6    650 m
7    550 m
8    200 m
9    550 m
Name: distance_skilift, dtype: object

In [33]:
df = df.rename(columns={"distance_skilift": "distance_skilift_meters"})

In [34]:
# "km" in der Spalte "distance_zentrum" in Meter umwandeln (nur falls als String vorhanden)
def km_to_meter_zentrum(val):
    if isinstance(val, str) and "km" in val:
        try:
            return int(float(val.replace("km", "").replace(",", ".").strip()) * 1000)
        except ValueError:
            return val
    return val

df["distance_zentrum"] = df["distance_zentrum"].apply(km_to_meter_zentrum)

In [35]:
df.head()

,web_scraper_order,web_scraper_start_url,price_raw,type,distance_zentrum,distance_skilift_meters,zimmer_grösse
0,1763925941-1,https://www.booking.com/searchresults.de.html?...,CHF 411,ferienwohnung,800,100 m,Apartment mit Blick auf die Berge
1,1763925941-2,https://www.booking.com/searchresults.de.html?...,CHF 134,ferienwohnung,900,600 m,Comfort Studio (2 Erwachsene)
2,1763925941-3,https://www.booking.com/searchresults.de.html?...,CHF 120,ferienwohnung,1400,1200,Zweibettzimmer
3,1763925941-4,https://www.booking.com/searchresults.de.html?...,CHF 199,ferienwohnung,800,300 m,Standard Doppelzimmer
4,1763925941-5,https://www.booking.com/searchresults.de.html?...,CHF 285,ferienwohnung,1200,100 m,Executive Doppelzimmer


In [36]:
df = df.rename(columns={"distance_zentrum": "distance_zentrum_meters"})

In [37]:
# Speichern (lokaler data-Ordner)
df.to_csv("data/Davos_combined_clean.csv", index=False)

In [38]:
# ensure df is available in the kernel — if not, try to load from the cleaned CSV
if "df" not in globals():
    if "pd" not in globals():
        import pandas as pd
    try:
        df = pd.read_csv("data/Davos_combined_clean.csv")
    except FileNotFoundError:
        raise FileNotFoundError("data/Davos_combined_clean.csv not found — run earlier cells to create it or place the file in the data/ folder")

# entferne das abschliessende "m" (inkl. optionaler Leerzeichen / NBSPs) und konvertiere falls möglich zu Integer
clean = (
    df["distance_zentrum_meters"]
    .astype(str)
    .str.replace("\u00A0", " ", regex=False)         # NBSP normalisieren
    .str.replace(r"\s*m\s*$", "", regex=True, case=False)  # entfernt "m" am Ende (z.B. "100 m" / "100m")
    .str.strip()
)

# 'nan' / leere Strings als NA behandeln und zu numerisch konvertieren
clean = clean.replace({"nan": pd.NA, "": pd.NA})
df["distance_zentrum_meters"] = pd.to_numeric(clean, errors="coerce").astype("Int64")

# speichern und kurze Vorschau
df.to_csv("data/Davos_combined_clean.csv", index=False)
df["distance_zentrum_meters"].head(10)

0     800
1     900
2    1400
3     800
4    1200
5     700
6     450
7    1300
8     900
9     700
Name: distance_zentrum_meters, dtype: Int64

In [39]:
# Klassifiziere "zimmer_grösse" weiter: Typ (apartment/zimmer/studio/chalet/other),
# Anzahl Schlafzimmer/Betten (wenn erkennbar), Personen-Angabe (z.B. "(2 Erwachsene)"), Balkon-Flag
s = df["zimmer_grösse"].astype(str).str.replace("\u00A0", " ", regex=False).str.strip().str.lower()

# Erkenne Typ
def _detect_typ(txt):
    if pd.isna(txt) or txt == "" or txt == "nan":
        return pd.NA
    if any(k in txt for k in ("apartment", "ap. ", "apt", "apart")):
        return "apartment"
    if "studio" in txt:
        return "studio"
    if "chalet" in txt:
        return "chalet"
    if any(k in txt for k in ("suite", "loft")):
        return "suite"
    # Zimmer / Bett Varianten
    if any(k in txt for k in ("doppelzimmer", "einzelzimmer", "zweibett", "dreibett", "bett", "zimmer")):
        return "zimmer"
    return "other"

df["zimmer_typ"] = s.apply(_detect_typ)

# Extrahiere explizite Zahlen (z.B. "2 Schlafzimmer", "1 Schlafzimmer", "3-Bett", "2 Betten")
num_from_digits = s.str.extract(r'(\d+)\s*(?:schlafzimmer|schlafzimmer|zimmer|bett|betten)')[0]
# Fallback: Wörter wie "einzel", "zweibett", "dreibett"
word_map = {"einzel": 1, "zweibett": 2, "dreibett": 3, "drei-bett": 3, "zweibettzimmer": 2}
num_from_words = s.replace(word_map, regex=True).str.extract(r'(\d+)', expand=False)

# choose numeric if present, else from words; convert to Int64
num_combined = num_from_digits.fillna(num_from_words)
df["zimmer_count"] = pd.to_numeric(num_combined, errors="coerce").astype("Int64")


# Balkonsignal
df["has_balkon"] = s.str.contains("balkon", na=False)

# Speichern und Vorschau
df.to_csv("data/Davos_combined_clean.csv", index=False)
df[["zimmer_grösse", "zimmer_typ", "zimmer_count", "has_balkon"]].head(15)

,zimmer_grösse,zimmer_typ,zimmer_count,has_balkon
0,Apartment mit Blick auf die Berge,apartment,<NA>,False
1,Comfort Studio (2 Erwachsene),studio,2,False
2,Zweibettzimmer,zimmer,<NA>,False
3,Standard Doppelzimmer,zimmer,<NA>,False
4,Executive Doppelzimmer,zimmer,<NA>,False
5,Einzelzimmer,zimmer,<NA>,False
6,Standard Einzelzimmer,zimmer,<NA>,False
7,Chalet,chalet,<NA>,False
8,Deluxe Apartment,apartment,<NA>,False
9,Apartment mit 2 Schlafzimmern,apartment,2,False


In [40]:
# sicherstellen, dass df vorhanden ist
if "df" not in globals():
    df = pd.read_csv("data/Davos_combined_clean.csv")

# nur fehlende zimmer_count füllen, falls "doppelzimmer" oder "einzelzimmer" in zimmer_grösse steht
mask_missing = df["zimmer_count"].isna()
text = df["zimmer_grösse"].astype(str).str.lower()

df.loc[mask_missing & text.str.contains("doppelzimmer", na=False), "zimmer_count"] = 2
df.loc[mask_missing & text.str.contains("einzelzimmer", na=False), "zimmer_count"] = 1

# sicherstellen, dass Spalte wieder als nullable Int gespeichert wird
df["zimmer_count"] = df["zimmer_count"].astype("Int64")

# speichern und kurze Vorschau
df.to_csv("data/Davos_combined_clean.csv", index=False)
df[["zimmer_grösse", "zimmer_count"]].head(20)

,zimmer_grösse,zimmer_count
0,Apartment mit Blick auf die Berge,<NA>
1,Comfort Studio (2 Erwachsene),2
2,Zweibettzimmer,<NA>
3,Standard Doppelzimmer,2
4,Executive Doppelzimmer,2
5,Einzelzimmer,1
6,Standard Einzelzimmer,1
7,Chalet,<NA>
8,Deluxe Apartment,<NA>
9,Apartment mit 2 Schlafzimmern,2


In [41]:
# rename column in the main df (and in related frames if present), then save
if "df" not in globals():
    df = pd.read_csv("data/Davos_combined_clean.csv")

if "zimmer_grösse" in df.columns:
    df.rename(columns={"zimmer_grösse": "zimmer_description"}, inplace=True)


    df.to_csv("data/Davos_combined_clean.csv", index=False)
    df[["zimmer_description"]].head(10)


In [42]:
# Entferne die beiden Spalten "web_scraper_order" und "web_scraper_start_url" aus df (falls vorhanden) und speichere
cols_to_drop = ["web_scraper_order", "web_scraper_start_url"]
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

df.to_csv("data/Davos_combined_clean.csv", index=False)
df.head()

,price_raw,type,distance_zentrum_meters,distance_skilift_meters,zimmer_description,zimmer_typ,zimmer_count,has_balkon
0,CHF 411,ferienwohnung,800,100 m,Apartment mit Blick auf die Berge,apartment,<NA>,False
1,CHF 134,ferienwohnung,900,600 m,Comfort Studio (2 Erwachsene),studio,2,False
2,CHF 120,ferienwohnung,1400,1200,Zweibettzimmer,zimmer,<NA>,False
3,CHF 199,ferienwohnung,800,300 m,Standard Doppelzimmer,zimmer,2,False
4,CHF 285,ferienwohnung,1200,100 m,Executive Doppelzimmer,zimmer,2,False


In [43]:
import sqlite3
import pandas as pd

# 1. SQLite-Verbindung öffnen (DB wird automatisch erstellt falls sie nicht existiert)
conn = sqlite3.connect("data/Davos_Ferienwohnung_Hotel.db")

# 2. Cleaned CSV laden
df = pd.read_csv("data/Davos_combined_clean.csv")

# 3. DataFrame in die SQLite-Datenbank schreiben
df.to_sql("ferienwohnung_hotel", conn, if_exists="replace", index=False)

# 4. Test: Daten aus DB wieder lesen
df_sql = pd.read_sql("SELECT * FROM ferienwohnung_hotel LIMIT 10;", conn)
print(df_sql)

# 5. Verbindung schliessen
conn.close()


  price_raw           type  distance_zentrum_meters distance_skilift_meters  \
0   CHF 411  ferienwohnung                      800                   100 m   
1   CHF 134  ferienwohnung                      900                   600 m   
2   CHF 120  ferienwohnung                     1400                    1200   
3   CHF 199  ferienwohnung                      800                   300 m   
4   CHF 285  ferienwohnung                     1200                   100 m   
5   CHF 165  ferienwohnung                      700                   500 m   
6   CHF 198  ferienwohnung                      450                   650 m   
7  CHF 1170  ferienwohnung                     1300                   550 m   
8   CHF 464  ferienwohnung                      900                   200 m   
9   CHF 679  ferienwohnung                      700                   550 m   

                  zimmer_description zimmer_typ  zimmer_count  has_balkon  
0  Apartment mit Blick auf die Berge  apartment       